# Import numpy and pandas

In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost

## Preparing dataset

In [26]:
# Load the Titanic training dataset
train_data = pd.read_csv("data/train.csv")

# Display the first 10 rows of the dataset for a better overview
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [27]:
# Load the Titanic testing dataset
test_data = pd.read_csv("data/test.csv")
test_data.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


## Calculating women and men survived rate

In [28]:
women_survived = train_data.loc[train_data.Sex == 'female', "Survived"]
rate_women = (women_survived.mean() * 100) if len(women_survived) > 0 else 0
print(f"% of women who survived: {rate_women:.2f}%")

% of women who survived: 74.20%


In [29]:
men_survived = train_data.loc[train_data.Sex == 'male', "Survived"]
rate_men = (men_survived.mean() * 100) if len(men_survived) > 0 else 0
print(f"% of men who survived: {rate_men:.2f}%")

% of men who survived: 18.89%


## Ready for model training

In [30]:
# Load the data
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

# 1. Handling Missing Values
# For Age: Impute with median, as age distribution is skewed
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].median())

# For Embarked: Impute with the most frequent value (mode)
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])

# For Fare: Impute with median for test data if it's missing
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())

# 2. Feature Engineering
# Create a new feature "FamilySize" as SibSp + Parch + 1 (for the individual)
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1

# Create a new feature "IsAlone" from FamilySize (1 if FamilySize is 1, else 0)
train_data['IsAlone'] = (train_data['FamilySize'] == 1).astype(int)
test_data['IsAlone'] = (test_data['FamilySize'] == 1).astype(int)

# Extract Title from Name (e.g., Mr., Mrs., Miss., etc.)
train_data['Title'] = train_data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
test_data['Title'] = test_data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

# Group less frequent titles into "Other"
title_mapping = {'Mr': 'Mr', 'Mrs': 'Mrs', 'Miss': 'Miss', 'Master': 'Master', 'Dr': 'Other', 'Rev': 'Other', 'Mlle': 'Other', 'Col': 'Other', 'Major': 'Other', 'Capt': 'Other', 'Sir': 'Other', 'Lady': 'Other', 'Jonkheer': 'Other', 'Countess': 'Other', 'Dona': 'Other'}
train_data['Title'] = train_data['Title'].map(title_mapping)
test_data['Title'] = test_data['Title'].map(title_mapping)

# 3. Encoding Categorical Variables
# Convert 'Sex' into binary variable (0 = male, 1 = female)
train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})

# Convert 'Embarked' into categorical variable using one-hot encoding
train_data = pd.get_dummies(train_data, columns=['Embarked'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Embarked'], drop_first=True)

# Convert 'Title' into categorical variable using one-hot encoding
train_data = pd.get_dummies(train_data, columns=['Title'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Title'], drop_first=True)

# 4. Dropping Unnecessary Columns
# Drop columns like 'Name', 'Ticket', 'Cabin', which are not needed directly
train_data = train_data.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_data = test_data.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# 5. Preparing Features and Target
y = train_data['Survived']  # Target variable
X = train_data.drop('Survived', axis=1)  # Features

# For test data, we don't have the target variable 'Survived', so we only prepare X_test
X_test = test_data


# Now, X is ready for model training, and X_test is ready for making predictions

##  Train Model

In [31]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Initialize the XGBoost model
model = XGBClassifier(n_estimators=100, random_state=1)

# Train the model on the training data
model.fit(X, y)

# Make predictions on the training data
train_predictions = model.predict(X)

# Calculate the accuracy of the model on the training set
train_accuracy = accuracy_score(y, train_predictions)
print(f"Model training accuracy: {train_accuracy * 100:.2f}%")

# Make predictions on the test set
test_predictions = model.predict(X_test)

# Create the output dataframe for submission
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': test_predictions})

# Save predictions to CSV file
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Model training accuracy: 99.89%
Your submission was successfully saved!
